In [1]:
from langchain import hub
from langchain_openai import ChatOpenAI

prompt = hub.pull('bsorrentino/image_to_plantuml')

print( prompt.format() )


Translate diagram in image in a plantUML script following rules below:

1. every rectangle or icon must be translate in plantuml rectangle element with related label if any
2. every rectangle that contains other elements must be translated in plantuml rectangle {}  element

answer with only plantuml script 


In [2]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.schema.messages import HumanMessage


def converImageToDiagram(image_url_or_data: str):

    prompt = hub.pull('bsorrentino/image_to_plantuml').format()
    
    openai = ChatOpenAI(model="gpt-4-vision-preview",
        max_tokens=2000,
        temperature=0.5
    )

    response = openai.invoke([
        HumanMessage(content=[
                {
                    "type": "text", 
                    "text": prompt
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": image_url_or_data
                    },
                },
            ]
        )
    ])

    return response.content



In [3]:
import gradio as gr
import base64, io
import PIL
from os import path 

def process_image(image: PIL.Image.Image):

    try:
        in_mem_file = io.BytesIO()
    
        image.save( in_mem_file, format = image.format if image.format is not None else 'PNG' )
        encoded_string = base64.b64encode(in_mem_file.getvalue())
        
        imageData = "data:image/png;base64," + encoded_string.decode('utf-8')

        result =  converImageToDiagram(imageData) 

        print( result )

        return result

    except Exception as e:
        return "Error: " + str(e)


demo = gr.Interface(
    fn=process_image,
    inputs=[
        gr.Image(type="pil")
    ],
    outputs=[
        gr.Markdown()
    ],
    # flagging_options=["blurry", "incorrect", "other"],
    examples=[
        path.join( "..",  "docs", "img_p3_1.png" )
    ],
)

# if __name__ == "__main__":
#     demo.launch()

demo.launch()

/Users/bsorrentino/WORKSPACES/GITHUB.me/AI/langchain-labs/langchain-labs#main/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


```plantuml
@startuml

rectangle "Event Stream" as ES

rectangle SP {
    rectangle "Preprocessing" as Pre
    rectangle "LLM Application" as LLMApp
    rectangle "Postprocessing" as Post
}

rectangle "Output" as Out
rectangle "Observability\n(Metrics, Logs, etc.)" as Obs

rectangle "LLM Service\n(eg., OpenAI)" as LLMServ
rectangle "LLM Tracing\n(eg., LangSmith)" as LLMTrac

ES -right-> Pre
Pre -right-> LLMApp
LLMApp -right-> Post
Post -right-> Out

LLMApp -down-> LLMServ
LLMApp -down-> LLMTrac
LLMApp -up-> Obs

note right of SP
  Stream Processor (eg., Kafka Consumer, Celery, RabbitMQ, SQS)
end note

@enduml
```
